In [1]:
import os
os.getcwd()

'C:\\Users\\julian.auriac\\DocLoc\\github\\BigDataHadoopSparkDaskCourse\\TPs\\Project\\Project'

In [2]:
!pip install imageio
import imageio
import os
import numpy as np

In [3]:
from dask.distributed import Client
import dask.dataframe as dd
from dask import compute
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import dask.array as da

In [4]:
def readImg(path):
    img = imageio.imread(path)
    im = np.array(img,dtype='uint8')
    nb_partitions = 8
    chunk_size = [img.shape[1]//8, img.shape[1], img.shape[2]]
    imgReturn = da.from_array(im, chunks=chunk_size)
    return imgReturn

def writeImg(path,buf):
    imageio.imwrite(path,buf)

In [5]:
#import dask_image.imread
#x = dask_image.imread.imread('./data/lena_noisy.jpg')
#x

In [6]:
data_dir="./data/"
file = os.path.join(data_dir,'lena_noisy.jpg')
img_buf=readImg(file)
#import dask_image.imread
#img_buf = dask_image.imread.imread('./data/lena_noisy.jpg')
img_buf
nx=img_buf.shape[0]
ny=img_buf.shape[1]

###########################################################################
#
# SPLIT IMAGES IN NB_PARTITIONS PARTS
nb_partitions = 8
print("NB PARTITIONS : ",nb_partitions)
data=[]
begin=0
block_size=nx/nb_partitions
for ip in range(nb_partitions):
    end=min(begin+block_size,nx)
    data.append([ip,begin,end,img_buf])
    begin=end


###########################################################################
#
# CREATE DASK CLIENT
client = Client(n_workers=8)
#from dask.distributed import LocalCluster, Client
#cluster = LocalCluster()
#client = Client(cluster)

NB PARTITIONS :  8


def part_median_filter(local_data):
    print(local_data)
    part_id = local_data[0]
    first   = local_data[1]
    end     = local_data[2]
    buf     = local_data[3]
    nx=buf.shape[0]
    ny=buf.shape[1]
    
    index = (part_id*ny//nb_partitions)
    if part_id == 0:
        temp = img_buf[index:(index)+ny//nb_partitions+1].copy()
    elif part_id == nb_partitions-1:
        temp = img_buf[index-1:(index)+ny//nb_partitions].copy()
    else:
        temp = img_buf[index-1:(index)+ny//nb_partitions+1].copy()

    new_buf = np.zeros(temp.shape)

    for i in range(0,temp.shape[0]-1):
        for j in range(0,temp.shape[1]-1):
            new_buf[i,j]=np.median([temp[i,j,:],temp[i+1,j,:],temp[i,j+1,:],temp[i+1,j+1,:],temp[i-1,j,:],temp[i,j-1,:],temp[i-1,j-1,:],temp[i+1,j-1,:],temp[i-1,j+1,:]],axis=0)
        new_buf[i,temp.shape[1]-1]=np.median([temp[i,j,:],temp[i,j+1,:],temp[i-1,j,:],temp[i,j-1,:],temp[i-1,j-1,:],temp[i-1,j+1,:]],axis=0)
        
    
    if part_id == 0:
        new_buf = new_buf[:-1].copy()
    elif part_id == nb_partitions-1:
        new_buf = new_buf[1:].copy()
        for j in range(0,temp.shape[1]-1):
            new_buf[ny//nb_partitions-1,j]=np.median([temp[i,j,:],temp[i+1,j,:],temp[i-1,j,:],temp[i,j-1,:],temp[i-1,j-1,:],temp[i+1,j-1,:]],axis=0)
    else:
        new_buf = new_buf[1:-1].copy()
    
    return new_buf #part_id,

In [7]:
def part_median_filter(local_img):
    nx=local_img.shape[0]
    ny=local_img.shape[1]
    
    ##########################################
    #
    # TODO COMPUTE MEDIAN FILTER
    #
    for i in range(nx):
        for j in range(ny):
            median_list = []
            
            if i != 0 and j != 0:
                median_list.append(local_img[i-1,j-1])
            if i != 0:
                median_list.append(local_img[i-1,j])
            if i != 0 and j != ny-1:
                median_list.append(local_img[i-1,j+1])
            if j != 0:
                median_list.append(local_img[i,j-1])
                
            median_list.append(local_img[i,j])
            
            if j != ny-1:
                median_list.append(local_img[i,j+1])
            if i != nx-1 and j != 0:
                median_list.append(local_img[i+1,j-1])
            if i != nx-1:
                median_list.append(local_img[i+1,j])
            if i != nx-1 and j != ny-1:
                median_list.append(local_img[i+1,j+1])
            
            if 1 == 2:
                local_img[i, j] = np.median(median_list)
            else:
                local_img[i, j, :] = np.median(median_list, axis=0)
    ##########################################
    #
    # RETURN LOCAL IMAGE PART
    #
    return local_img

In [8]:
###########################################################################
#
# PARALLEL MEDIAN FILTER COMPUTATION
#result_rdd = data_rdd.map(part_median_filter)
#result_data = result_rdd.collect()

import joblib
with joblib.parallel_backend('dask'):
    #result_rdd = data.map(part_median_filter)
    #result_data = result_rdd.compute()
    new_img_buf = img_buf.map_overlap(part_median_filter, depth=(1, img_buf.shape[1])).compute()

#new_img_buf = img_buf.map_overlap(part_median_filter, depth=(1, img_buf.shape[1])).compute()

In [9]:
###########################################################################
#
# COMPUTE NEW IMAGE RESULTS FROM RESULT RDD
# TODO
#new_img_buf = []
#for part in result_data:
#    new_img_buf.append(part[1])

#new_img_buf = np.vstack(new_img_buf)
    
print('CREATE NEW PICTURE FILE')
filter_file = os.path.join(data_dir,'lena_filter.jpg')
writeImg(filter_file,new_img_buf)

CREATE NEW PICTURE FILE


In [10]:
client.close()